In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from z3 import Int, Solver, sat
from spytial import diagram
from spytial.annotations import orientation, attribute

# Define a simple Z3 problem
x = Int('x')
y = Int('y')
s = Solver()
s.add(x + y == 10, x > 0, y > 0)

if s.check() == sat:  # Compare with z3.sat
    z3_model = s.model()
    print("Z3 Model:")
    
    diagram(z3_model)
else:
    print(s.check())
    print("No solution found.")

Z3 Model:


# But is this useful? Not really

You can register a DataInstanceBuilder for Z3 that provides some directions on HOW things should be translated to boxes and arrows.



In [4]:
from spytial.provider_system import DataInstanceProvider, data_provider
from z3 import ModelRef

@data_provider(priority=10)
class ArithmeticTheoryProvider(DataInstanceProvider):
    """Custom provider for Z3 models in the arithmetic theory."""

    def can_handle(self, obj):
        return isinstance(obj, ModelRef)  # Check if the object is a Z3 model

    def provide_atoms_and_relations(self, obj, walker_func):
        atoms = []
        relations = []

        # Add variables as atoms
        for decl in obj.decls():
            try:
                var_name = decl.name()
                var_expr = decl()  # Convert the declaration into a Z3 expression
                var_value = obj.eval(var_expr, model_completion=True)  # Evaluate the expression
                atom_id = walker_func._get_id(var_name)
                atom = {
                    "id": atom_id,
                    "type": "ArithmeticVariable",
                    "label": f"{var_name} = {var_value}"
                }
                atoms.append(atom)
            except Exception as e:
                print(f"Skipping declaration {decl}: {e}")

        # Add constraints as relations
        # Example: Add relations for constraints (extend this for real constraints)
        for decl in obj.decls():
            if decl.name() == "x":  # Example: Add a relation for x
                x_id = walker_func._get_id("x")
                y_id = walker_func._get_id("y")
                relations.append(("constraint", x_id, y_id))

        return atoms, relations

# Use the custom ArithmeticTheoryProvider to visualize the Z3 model
# The diagram function will now use the registered provider for better DEFAULT visualization

diagram(z3_model)


Skipping declaration y: b'ast is not an expression'
Skipping declaration x: b'ast is not an expression'
Error during data instance building: list indices must be integers or slices, not str
Object: [y = 1, x = 9]
